[Official documentation on chat from LangChain](https://python.langchain.com/en/latest/modules/models/chat/getting_started.html)

In [1]:
!pip install langchain
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.4/969.4 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.7 MB/s eta 0:00:00


In [2]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [3]:
OPENAI_API_KEY = ''

### 0. Simple Input/Output Still works

You just need to pass it as a message instead. More on this in a second.

In [4]:
chat = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [5]:
message = [HumanMessage(content="What is the name of the most populous state in the USA?")]

chat(message)

AIMessage(content='California.', additional_kwargs={}, example=False)

### 1. Chat Messages

* HumanMessage: A message sent from the perspective of the human
* AIMessage: A message sent from the perspective of the AI the human is interacting with
* SystemMessage: A message setting the objectives the AI should follow
* ChatMessage: A message allowing for arbitrary setting of role. You won’t be using this too much

In [6]:
messages = [
    SystemMessage(content="Say the opposite of what the user says"),
    HumanMessage(content="I love programming.")
]
chat(messages)

AIMessage(content='I hate programming.', additional_kwargs={}, example=False)

In [7]:
messages = [
    SystemMessage(content="Say the opposite of what the user says"),
    HumanMessage(content="I love programming."),
    AIMessage(content='I hate programming.'),
    HumanMessage(content="The moon is out")
]
chat(messages)

AIMessage(content='The moon is not out.', additional_kwargs={}, example=False)

In [8]:
messages = [
    SystemMessage(content="Say the opposite of what the user says"),
    HumanMessage(content="I love programming."),
    AIMessage(content='I hate programming.'),
    HumanMessage(content="What is the first thing that I said?")
]
chat(messages)

AIMessage(content='You said "I love programming."', additional_kwargs={}, example=False)

#### Batch Messages

In [9]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful word machine that creates an alliteration using a base word"),
        HumanMessage(content="Base word: Apple")
    ],
    [
        SystemMessage(content="You are a helpful word machine that creates an alliteration using a base word"),
        HumanMessage(content="Base word: Dog")
    ],
]
chat.generate(batch_messages)

LLMResult(generations=[[ChatGeneration(text='Amazingly Appetizing Apples', generation_info=None, message=AIMessage(content='Amazingly Appetizing Apples', additional_kwargs={}, example=False))], [ChatGeneration(text='Dandy doggy dug diligently.', generation_info=None, message=AIMessage(content='Dandy doggy dug diligently.', additional_kwargs={}, example=False))]], llm_output={'token_usage': {'prompt_tokens': 64, 'completion_tokens': 13, 'total_tokens': 77}, 'model_name': 'gpt-3.5-turbo'})

### Prompt Templates
With one or more `MessagePromptTemplates` you can build a `ChatPromptTemplate`

In [10]:
# Make SystemMessagePromptTemplate
prompt=PromptTemplate(
    template="Propose creative ways to incorporate {food_1} and {food_2} in the cuisine of the users choice.",
    input_variables=["food_1", "food_2"]
)

system_message_prompt = SystemMessagePromptTemplate(prompt=prompt)

In [11]:
# Output of system_message_prompt
system_message_prompt.format(food_1="Bacon", food_2="Shrimp")

SystemMessage(content='Propose creative ways to incorporate Bacon and Shrimp in the cuisine of the users choice.', additional_kwargs={})

In [12]:
# Make HumanMessagePromptTemplate
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [13]:
# Create ChatPromptTemplate: Combine System + Human
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chat_prompt

ChatPromptTemplate(input_variables=['text', 'food_1', 'food_2'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['food_1', 'food_2'], output_parser=None, partial_variables={}, template='Propose creative ways to incorporate {food_1} and {food_2} in the cuisine of the users choice.', template_format='f-string', validate_template=True), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='{text}', template_format='f-string', validate_template=True), additional_kwargs={})])

In [14]:
chat_prompt_with_values = chat_prompt.format_prompt(food_1="Bacon", \
                                                   food_2="Shrimp", \
                                                   text="I really like food from Germany.")

chat_prompt_with_values.to_messages()

[SystemMessage(content='Propose creative ways to incorporate Bacon and Shrimp in the cuisine of the users choice.', additional_kwargs={}),
 HumanMessage(content='I really like food from Germany.', additional_kwargs={}, example=False)]

In [15]:
response = chat(chat_prompt_with_values.to_messages()).content
print (response)

One way to incorporate bacon and shrimp in German cuisine is to make a classic dish called "Spätzle" which are soft egg noodles typically served with gravy or cheese. Here's a recipe to add bacon and shrimp to the dish:

Ingredients:
- 2 cups Spätzle noodles
- 4 slices of bacon, chopped
- 1/2 lb. shrimp, peeled and deveined
- 1/2 onion, diced
- 2 cloves garlic, minced
- 1/2 cup heavy cream
- 1/2 cup chicken broth
- Salt and pepper, to taste
- Fresh parsley, chopped

Instructions:
1. Cook Spätzle noodles according to package directions. Drain and set aside.
2. In a large skillet, cook chopped bacon over medium heat until crispy. Remove bacon from skillet and set aside.
3. Add diced onion and minced garlic to the bacon grease in the skillet and cook until onion is soft and fragrant.
4. Add peeled and deveined shrimp to the skillet and cook until pink and cooked through.
5. Pour chicken broth and heavy cream into the skillet and bring to a simmer.
6. Add cooked Spätzle noodles to the skil

#### With Streaming

In [16]:
from langchain.callbacks.base import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
chat = ChatOpenAI(openai_api_key=OPENAI_API_KEY, streaming=True, callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]), verbose=True, temperature=0)

resp = chat(chat_prompt_with_values.to_messages())

ImportError: ignored